<a href="https://colab.research.google.com/github/jiwoong2/deeplearning/blob/main/time_layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNN(Recurrent Neural Network)


In [ ]:
class RNN:
    def __init__(self, Wx, Wh, b):
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.cache = None

    def forward(self, x, h_prev):
        Wx, Wh, b = self.params
        t = np.dot(h_prev, Wh) + np.dot(x, Wx) + b
        h_next = np.tanh(t)

        self.cache = (x, h_prev, h_next)
        return h_next

    def backward(self, dh_next): # dh_next는 흘러들어온 미분
        Wx, Wh, b = self.params
        x, h_prev, h_next = self.cache

        dt = dh_next * (1 - h_next ** 2) # 활성화 함수 tanh에 대한 미분.(1-y^2)
        db = np.sum(dt, axis=0) # repeat node에 대한 역전파.
        dWh = np.dot(h_prev.T, dt) # dot 연산에 대한 역전파
        dh_prev = np.dot(dt, Wh.T) # dot 연산에 대한 역전파
        dWx = np.dot(x.T, dt) # dot 연산에 대한 역전파
        dx = np.dot(dt, Wx.T) # dot에 연산에 대한 역전파

        self.grads[0][...] = dWx
        self.grads[1][...] = dWh
        self.grads[2][...] = db

        return dx, dh_prev

# TimeRNN(미니배치, TruncatedBPTT)

In [ ]:
class TimeRNN:
    def __init__(self, Wx, Wh, b, stateful=False): # stateful은 TruncatedBPTT의 시간블록의 출력인 히든스테이트를 다음 시간블록으로 전달할지 말지 결정한다.
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.layers = None

        self.h, self.dh = None, None
        self.stateful = stateful

    def forward(self, xs):
        Wx, Wh, b = self.params
        N, T, D = xs.shape # 배치 크기, 시간블록 크기, embedding층을 거친 벡터 차원.
        D, H = Wx.shape # data 는 D, D를 은닉층 H 으로 보내는 Weight matirx.

        self.layers = [] # RNN 시간블록을 저장할 리스트.
        hs = np.empty((N, T, H), dtype='f') # 3차원 텐서 생성. empty는 랜덤한 값을 생성함. 히든스테이트들을 저장할 것.

        if not self.stateful or self.h is None:
            self.h = np.zeros((N, H), dtype='f') # 히든스테이를 다음 블록으로 전달하지 않을 경우 제로행렬을 전달.

        for t in range(T): # T는 타임사이즈.
            layer = RNN(*self.params) # 브라켓을 떼고 RNN인슨턴스 생성에 필요한 파라미터를 입력.
            self.h = layer.forward(xs[:, t, :], self.h) # 시간블록 선택(data), 히든스테이트 선택.
            hs[:, t, :] = self.h
            self.layers.append(layer)

        return hs # 히든 스테이트들의 모임을 반환.

    def backward(self, dhs):
        Wx, Wh, b = self.params
        N, T, H = dhs.shape # 흘러들어온 미분의 shape.
        D, H = Wx.shape

        dxs = np.empty((N, T, D), dtype='f')
        dh = 0 # dh는 다음 시간블록에서 흘러들어온 미분.
        grads = [0, 0, 0] # gradient를 저장할 리스트.
        for t in reversed(range(T)): # 역전파이므로 마지막 시간블록부터 시작.
            layer = self.layers[t]
            dx, dh = layer.backward(dhs[:, t, :] + dh) # 흘러들어온 두 미분을 합한다.
            dxs[:, t, :] = dx

            for i, grad in enumerate(layer.grads): # gardient 저장.
                grads[i] += grad

        for i, grad in enumerate(grads):
            self.grads[i][...] = grad
        self.dh = dh

        return dxs

    def set_state(self, h):
        self.h = h

    def reset_state(self):
        self.h = None